Data acquisition

In [1]:
import sys
sys.path.append('/home/ppiper/Dropbox/local/github/frog')

from frog.datahandler import HDF5Handler, sliceDataAlongAxis

In [2]:
lfdh = HDF5Handler(
    datapath = '/home/ppiper/Dropbox/local/ihtc_nozzle/data/doe_lhs_multirun_N200/Q1D_transposed.hdf5', 
    datasets = ['p.txt', 'T.txt', 'M.txt']
)

In [3]:
hfdh = HDF5Handler(
    datapath='/home/ppiper/Dropbox/local/ihtc_nozzle/data/doe_lhs_multirun_N200/SU2_fluid_transposed.hdf5',
    datasets = ['Pressure', 'Temperature', 'Mach', 'Heat_Flux'],
)

In [4]:
lf_train, lf_validation, lf_test = sliceDataAlongAxis(lfdh[:], [0.8, 0.1, 0.1], axis=0)

hf_train, hf_validation, hf_test = sliceDataAlongAxis(hfdh[:], [0.8, 0.1, 0.1], axis=0)

Oder reduction

In [5]:
from frog.rom import SVD

In [6]:
lf_svd = SVD(lf_train)
#lf_svd.normalize(bounds=[-1,1])
#lf_svd.subtractMean()
lf_svd.SVD()
#lf_rank = lf_svd.findRank(99.9)
lf_rank = 5
lf_svd.setRank(lf_rank)

(array([[-9.39504560e-02,  1.33980383e-01,  1.21190822e-01,
          4.14083328e-02,  1.26533757e-01],
        [-9.39435303e-02,  1.26863635e-01,  1.11584155e-01,
          3.68465146e-02,  1.14586236e-01],
        [-9.39492913e-02,  1.27620371e-01,  1.12606047e-01,
          3.71405906e-02,  1.13589947e-01],
        ...,
        [-1.94004550e-07,  5.73465260e-07, -4.13649422e-07,
         -1.53953151e-06,  3.04762960e-06],
        [-1.94801976e-07,  5.79262822e-07, -4.19444995e-07,
         -1.56367592e-06,  3.09164965e-06],
        [-1.95600946e-07,  5.85281923e-07, -4.25101234e-07,
         -1.58244265e-06,  3.20387159e-06]]),
 array([[1.52119502e+08, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
         0.00000000e+00],
        [0.00000000e+00, 6.30737698e+05, 0.00000000e+00, 0.00000000e+00,
         0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 4.21762467e+05, 0.00000000e+00,
         0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 3.28531

In [7]:
hf_svd = SVD(hf_train)
#hf_svd.normalize(bounds=[-1,1])
#hf_svd.subtractMean()
hf_svd.SVD()
#hf_rank = hf_svd.findRank(99.9)
hf_rank = 5
hf_svd.setRank(hf_rank)

(array([[-5.29248559e-03,  1.80142241e-03,  2.02395072e-03,
          2.48899329e-02,  5.91873319e-03],
        [-8.14568294e-04,  1.04426633e-05,  2.66895333e-04,
          1.36908590e-03, -6.48799025e-04],
        [-5.29248512e-03,  1.80146185e-03,  2.02523897e-03,
          2.48844856e-02,  5.92155522e-03],
        ...,
        [-1.45854558e-03, -5.76773924e-02,  3.22008815e-02,
          4.77928622e-02,  3.13115085e-02],
        [-1.45200824e-03, -5.79915203e-02,  3.32554837e-02,
          4.71292474e-02,  3.45257755e-02],
        [-1.79289909e-03, -6.64056073e-02,  2.40998626e-02,
          5.58533698e-02,  3.28244947e-02]]),
 array([[2.66536680e+09, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
         0.00000000e+00],
        [0.00000000e+00, 4.45497322e+07, 0.00000000e+00, 0.00000000e+00,
         0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 1.12451525e+06, 0.00000000e+00,
         0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 1.74429

Surrogate model (NN)

In [8]:
from frog.neuralnetwork import mlp
from frog.datahandler import sliceDataAlongAxis

2023-06-28 01:48:29.776181: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-28 01:48:29.967617: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [9]:
lf_train_L, lf_validation_L, lf_test_L = sliceDataAlongAxis(lf_svd.compress(lfdh[:].T), [0.8, 0.1, 0.1], axis=0)

hf_train_L, hf_validation_L, hf_test_L = sliceDataAlongAxis(hf_svd.compress(hfdh[:].T), [0.8, 0.1, 0.1], axis=0)

In [10]:
nn, history = mlp(
    inputs_train = lf_train_L, 
    outputs_train = hf_train_L, 
    inputs_validation = lf_validation_L,
    outputs_validation = hf_validation_L,
    layers = (lf_rank,hf_rank,5,10) ,
    fit_kwargs=dict(epochs=50, batch_size=16))

Epoch 1/50
10/10 [==============================] - 3s 59ms/step - loss: 9801608216969216.0000 - val_loss: 9802082810855424.0000
Epoch 2/50
10/10 [==============================] - 0s 9ms/step - loss: 9801608216969216.0000 - val_loss: 9802082810855424.0000
Epoch 3/50
10/10 [==============================] - 0s 11ms/step - loss: 9801608216969216.0000 - val_loss: 9802082810855424.0000
Epoch 4/50
10/10 [==============================] - 0s 10ms/step - loss: 9801608216969216.0000 - val_loss: 9802082810855424.0000
Epoch 5/50
10/10 [==============================] - 0s 11ms/step - loss: 9801608216969216.0000 - val_loss: 9802082810855424.0000
Epoch 6/50
10/10 [==============================] - 0s 16ms/step - loss: 9801608216969216.0000 - val_loss: 9802082810855424.0000
Epoch 7/50
10/10 [==============================] - 0s 11ms/step - loss: 9801608216969216.0000 - val_loss: 9802082810855424.0000
Epoch 8/50
10/10 [==============================] - 0s 13ms/step - loss: 9801608216969216.0000 - v

Inference

In [11]:
prediction = nn.predict(lf_test)


KeyboardInterrupt



In [ ]:
nn.evaluate(lf_test, hf_test)

In [ ]:
reconstructed = hf_svd.reconstruct(prediction)

In [ ]:
hfdh_train, hfdh_validation, hfdh_test = sliceDataAlongAxis(hfdh[:], [0.8, 0.1, 0.1], axis=0)

In [ ]:
import matplotlib.pyplot as plt

plt.plot(hfdh(reconstructed)[0,'Heat_Flux'])
plt.plot(hfdh(hfdh_test)[0,'Heat_Flux'])

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor
from sklearn.gaussian_process import GaussianProcessRegressor

from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np

class DataHandlerNormalization(BaseEstimator, TransformerMixin):
    def __init__(self,datahandler, bounds):
        self.bounds = bounds
        self.datahandler = datahandler

    def fit(self, X, y=None):
        self.mean = {}
        self.min = {}
        self.max = {}
        for dataset in self.datahandler.datasets:
            self.mean[dataset] = np.mean(self.datahandler(X)[dataset], axis=0)
            self.min[dataset] = np.min(self.datahandler(X)[dataset])
            self.max[dataset] = np.max(self.datahandler(X)[dataset])
    
    def transform(self, X, y=None):
        for dataset in self.datahandler.datasets:
            X[self.datahandler.get_index(dataset)] = X[self.datahandler.get_index(dataset)] - self.mean[dataset]
            X[self.datahandler.get_index(dataset)] = (self.bounds[1]-self.bounds[0]) * (X[self.datahandler.get_index(dataset)] - self.min[dataset]) / (self.max[dataset] - self.min[dataset]) + self.bounds[0]
        return X

    def inverse_transform(self, X, y=None):
        print('inverse_transform')
        for dataset in self.datahandler.datasets:
            X[self.datahandler.get_index(dataset)] = (self.max[dataset] - self.min[dataset]) * (X[self.datahandler.get_index(dataset)] - self.bounds[0]) / (self.bounds[1] - self.bounds[0]) + self.min[dataset]
            X[self.datahandler.get_index(dataset)] = X[self.datahandler.get_index(dataset)] + self.mean[dataset]
        return X

    def fit_transform(self, X, y=None):
        self.fit(X)
        return self.transform(X)


class Normalization(BaseEstimator, TransformerMixin):
    def __init__(self, bounds=[-1,1]):
        self.bounds = bounds

    def fit(self, X, y=None):
        self.mean = np.mean(X, axis=0)
        self.min = np.min(X)
        self.max = np.max(X)

    def fit_transform(self, X, y=None):
        self.fit(X)

        return self.transform(X)

    def transform(self, X, y=None):
        X = X - self.mean
        X = (self.bounds[1]-self.bounds[0]) * (X - self.min) / (self.max - self.min) + self.bounds[0]
        #X = np.nan_to_num(X, 0)
        return X

    def inverse_transform(self, X, y=None):
        print('inverse_transform')
        X = (self.max - self.min) * (X - self.bounds[0]) / (self.bounds[1] - self.bounds[0]) + self.min
        X = X + self.mean
        return X

X_rom = Pipeline([
    #('scaler', StandardScaler()),
    #('scaler', Normalization(bounds=[0,1])),
    ('scaler', DataHandlerNormalization(datahandler=hfdh, bounds=[0,1])),

    #('scaler', RobustScaler()),
    ('pca', PCA(n_components=5)),
])


y_rom = Pipeline([
    #('scaler', StandardScaler()),
    #('scaler', Normalization(bounds=[0,1])),
    ('scaler', DataHandlerNormalization(datahandler=hfdh, bounds=[0,1])),
    #('scaler', RobustScaler()),
    ('pca', PCA(n_components=5)),
])


surrogate = Pipeline([
    ('regression', LinearRegression())
])

surrogate = Pipeline([
    ('regression', GaussianProcessRegressor())
])

#surrogate = Pipeline([
#    ('regression', MLPRegressor(hidden_layer_sizes=(5,5,5), max_iter=1000))
#])

#from frog.neuralnetwork import get_model

#surrogate = Pipeline([
#    ('regression', get_model(5,5,5,10))
#])

#surrogate = get_model(5,5,1,5)

In [ ]:
class FR:
    def __init__(self, X_rom, y_rom, surrogate):
        self.X_rom = X_rom
        self.y_rom = y_rom
        self.surrogate = surrogate

    def fit(self, X, y, fit_kwargs : dict = {}):
        print('Performing ROM fit on X data')
        X = self.X_rom.fit_transform(X)
        print('Performing ROM fit on y data')
        y = self.y_rom.fit_transform(y)

        if 'validation_data' in fit_kwargs.keys():
            X_validation = fit_kwargs['validation_data'][0]
            y_validation = fit_kwargs['validation_data'][1]
            print('Performing ROM fit on X validation data')
            X_validation = self.X_rom.transform(X_validation)
            print('Performing ROM fit on y validation data')
            y_validation = self.y_rom.transform(y_validation)
            fit_kwargs['validation_data'] = (X_validation, y_validation)
            
        print('Performing surrogate model fit')
        self.surrogate.fit(
            X, 
            y, 
            **fit_kwargs
        )
    
    def predict(self, X):
        X_in = self.X_rom.transform(X)
        y_out = self.surrogate.predict(X_in)
        y_out = self.y_rom.inverse_transform(y_out)
        
        return y_out
           

In [ ]:
fr.fit(
    lf_train, 
    hf_train, 
    fit_kwargs = dict( epochs=50, batch_size=16, validation_data=(lf_validation, hf_validation))
)

Performing ROM fit on X data


IndexError: arrays used as indices must be of integer (or boolean) type

In [ ]:
fr.X_rom['scaler'].mean

In [ ]:
import matplotlib.pyplot as plt

plt.plot(hfdh(fr.predict(lf_test))[4,'Heat_Flux'])
#plt.plot(hfdh(fr.predict(lf_test[None,0]))[0,'Heat_Flux'])
plt.plot(hfdh(hf_test)[4,'Heat_Flux'])